<a href="https://colab.research.google.com/github/MiggieCS911/DrugRepuposingSGLT2i/blob/main/findOtherSubstancesbyModelSGLT2_show.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pickle

# load RFmodel
name => rfmodel

In [ ]:
filename = '/content/drive/MyDrive/AI for drug/block_SGLT2/results/RFmodelHighfeature_bioactivity_class.sav'
rfmodel = pickle.load(open(filename, 'rb'))

# load XGBoost

In [ ]:
!pip install xgboost==1.3.3
import xgboost as xgb

#load model
xgbmodel = xgb.XGBClassifier()
xgbmodel.load_model('/content/drive/MyDrive/AI for drug/block_SGLT2/results/xgb_cioactivity_class.json')

     |████████████████████████████████| 157.5 MB 32 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


# prepared function turn canonical smiles to fp
use PaDELpy

In [ ]:
!pip install padelpy

     |████████████████████████████████| 20.9 MB 273 kB/s 


In [ ]:
from padelpy import from_smiles

Use datafrom CHEMBLfiltered

In [ ]:
dfchem = pd.read_csv('/content/drive/MyDrive/AI for drug/block_SGLT2/data/chemBL/chembl_filtered_torchdrug.csv',
                     usecols = ['smiles'])

In [ ]:
dfchem.shape

(430710, 1)

In [ ]:
dfchem.head()

,smiles
0,CCCCCC/C=C\CCCCCCCc1cccc(O)c1C(=O)O
1,CC(=O)c1cc2c(cc1O)OC(C)(C)C=C2
2,CCCCCn1cc(C(=O)NC2C3CC4CC(C3)CC2C4)c(=O)c2ccccc21
3,O=C1NC(Cc2c[nH]c3ccccc23)C(=O)N2C(c3ccccc3)c3[...
4,N=C(N)NC(=O)c1nc(Cl)c(N2CCCCCC2)nc1N


use cananonical smiles from SGLT2-i

In [ ]:
dfsglt2 = pd.read_csv('/content/drive/MyDrive/AI for drug/block_SGLT2/data/sglt2i.csv',
                      usecols = ['canonical_smiles'])

In [ ]:
dfsglt2.shape

(1528, 1)

In [ ]:
dfsglt2.head()

,canonical_smiles
0,Cc1cc(O)c(C(=O)CCc2ccc3occc3c2)c(O[C@@H]2O[C@H...
1,Cc1cc(O[C@@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@H]...
2,Cc1cc(O[C@@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@H]...
3,CCc1cc(O[C@@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@H...
4,Cc1cc(O[C@@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@H]...


# step test
1. check not in dfsglt2
2. turn to descriptor
3. predict by model
4. if active -> 1 save canonical smiles in list
5. save list to csv

due to very long time when turn to fp --> we cut to 100 substances each time

In [ ]:
substance = dfchem.smiles.to_list()

In [ ]:
substance[:10]

['CCCCCC/C=C\\CCCCCCCc1cccc(O)c1C(=O)O',
 'CC(=O)c1cc2c(cc1O)OC(C)(C)C=C2',
 'CCCCCn1cc(C(=O)NC2C3CC4CC(C3)CC2C4)c(=O)c2ccccc21',
 'O=C1NC(Cc2c[nH]c3ccccc23)C(=O)N2C(c3ccccc3)c3[nH]c4ccccc4c3C[C@@H]12',
 'N=C(N)NC(=O)c1nc(Cl)c(N2CCCCCC2)nc1N',
 'CC(C)(C)OC(=O)NC(CC(=O)OCc1ccccc1)C(=O)NC(COCc1ccccc1)C(=O)NCC(=O)OCc1ccccc1',
 'O=C(O)/C(O)=C/C(=O)c1ccnc2ccccc12',
 'CC1=CC(C)(C)N=C(Nc2nc(C)c3cc(C)cc(C)c3n2)N1',
 'COc1cc(CNc2ccc(C(C)C)cc2)cc(OC)c1OC',
 'O=c1ccc2nccc3c4c(O)cccc4n1c23']

In [ ]:
sglt = dfsglt2.canonical_smiles.to_list()

In [ ]:
sglt[:4]

['Cc1cc(O)c(C(=O)CCc2ccc3occc3c2)c(O[C@@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@H]2O)c1',
 'Cc1cc(O[C@@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@H]2O)c2c(CCc3ccc4occc4c3)n[nH]c2c1',
 'Cc1cc(O[C@@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@H]2O)c2c(CCc3ccc4c(c3)CCO4)n[nH]c2c1',
 'CCc1cc(O[C@@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@H]2O)c2c(CCc3ccc4c(c3)CCO4)n[nH]c2c1']

In [ ]:
# 0 - 10,000
%%time
active_RF, prob_active_RF,inactive_RF,prob_inactive_RF =[],[],[],[]
active_xgb, prob_active_xgb,inactive_xgb,prob_inactive_xgb =[],[],[],[]

for i in range(len(substance[:10000])):

  #check not in known data
  if substance[i] not in sglt:

    #turn to fp
    fp = from_smiles(substance[i], fingerprints=True, descriptors=False)

    #turn orderdict to numpy array  -> input model
    input = np.array([v for k, v in fp.items()])

    #reshape input to fit to model
    input = input.reshape(1,-1)

    #predict by RFmodel
    res = rfmodel.predict(input)
    prob_res_0, prob_res_1 = rfmodel.predict_proba(input)[0] #to prob
    if res == 1:
      active_RF.append(substance[i])
      prob_active_RF.append(prob_res_1)#save prob predict active
    else:
      inactive_RF.append(substance[i])
      prob_inactive_RF.append(prob_res_1)#save prob predict active

    #predict by xgboost model
    res_xgb = xgbmodel.predict(input)
    prob_xgb_0, prob_xgb_1 = xgbmodel.predict_proba(input)[0]
    if res_xgb == 1:
      active_xgb.append(substance[i])
      prob_active_xgb.append(prob_xgb_1) #save prob predict active
    else:
      inactive_xgb.append(substance[i])
      prob_inactive_xgb.append(prob_xgb_1)#save prob predict active
  if i%1000 == 0:
    print(i)

#create and save dataframe
df_active_RF = pd.DataFrame(zip(active_RF,prob_active_RF),columns=['smiles','prob_active'])
df_inactive_RF = pd.DataFrame(zip(inactive_RF, prob_inactive_RF), columns=['smiles','prob_active'])
df_active_xgb = pd.DataFrame(zip(active_xgb, prob_active_xgb),columns=['smiles','prob_active'])
df_inactive_xgb = pd.DataFrame(zip(inactive_xgb, prob_inactive_xgb),columns=['smiles','prob_active'])

#save to csv files
df_active_RF.to_csv('/content/drive/MyDrive/AI for drug/block_SGLT2/data/active0_10000RF.csv',index=None)
df_inactive_RF.to_csv('/content/drive/MyDrive/AI for drug/block_SGLT2/data/inactive0_10000RF.csv',index=None)
df_active_xgb.to_csv('/content/drive/MyDrive/AI for drug/block_SGLT2/data/active0_10000XGB.csv',index=None)
df_inactive_xgb.to_csv('/content/drive/MyDrive/AI for drug/block_SGLT2/data/inactive0_10000XGB.csv',index=None)
print('***Finished 10,000***')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
***Finished 10,000***
CPU times: user 15min 46s, sys: 1min 28s, total: 17min 15s
Wall time: 15h 1min 2s
